In [6]:
    from fractions import Fraction

    def compress_arithmetic(sensor_data):
        compressed_data = {}
        for sensor, data in sensor_data.items():
            # Convert the data to a string
            data_string = str(data)

            # Calculate the cumulative probabilities of each symbol
            symbol_counts = {}
            for symbol in data_string:
                symbol_counts[symbol] = symbol_counts.get(symbol, 0) + 1
            total_symbols = len(data_string)
            cumulative_probabilities = {}
            current_sum = 0
            for symbol, count in symbol_counts.items():
                symbol_prob = Fraction(count, total_symbols)
                cumulative_probabilities[symbol] = (current_sum, current_sum + symbol_prob)
                current_sum += symbol_prob

            # Perform arithmetic coding
            low = 0
            high = 1
            encoded_data = []
            for symbol in data_string:
                symbol_low, symbol_high = cumulative_probabilities[symbol]
                range_size = high - low
                high = low + range_size * symbol_high
                low = low + range_size * symbol_low

                # Rescaling to prevent precision issues
                while (high < Fraction(1, 2)) or (low > Fraction(1, 2)):
                    if high <=Fraction(1, 2):
                        encoded_data.append(1)
                        high = 2 * high
                        low = 2 * low
                    else:
                        encoded_data.append(0)
                        high = 2 * (high - Fraction(1, 2))
                        low = 2 * (low - Fraction(1, 2))

            # Output the compressed data
            compressed_data[sensor] = encoded_data

        return compressed_data


    # Example usage:
    sensor_data = {
        "Engine Speed": 1330,
        "Throttle Position": 1234,
        "Vehicle Speed": 1610,
        "Oxygen Sensor": 101.1715,
        "Temperature Sensors": 1810,
        
    }

    compressed_data = compress_arithmetic(sensor_data)
    for sensor, data in compressed_data.items():
        print(sensor + ": " + "".join(str(bit) for bit in data))


Engine Speed: 1101
Throttle Position: 11100
Vehicle Speed: 101
Oxygen Sensor: 101110101001
Temperature Sensors: 101
